---
title: Audio processing pipeline
output-file: audio_processor.html
description: Loads raw audio streams
---

In [ ]:
#| default_exp audio_processor

In [ ]:
!mfa model download acoustic english_us_arpa
!mfa model download dictionary english_us_arpa

INFO - Saved model to f/home/jovyan/Documents/MFA/pretrained_models/acoustic/english_us_arpa.zip, you can now use
                english_us_arpa in place of acoustic paths in mfa commands.
INFO - Saved model to f/home/jovyan/Documents/MFA/pretrained_models/acoustic/english_us_arpa.zip, you can now use
                english_us_arpa in place of acoustic paths in mfa commands.
INFO - Saved model to f/home/jovyan/Documents/MFA/pretrained_models/dictionary/english_us_arpa.dict, you can now use
                english_us_arpa in place of dictionary paths in mfa commands.
INFO - Saved model to f/home/jovyan/Documents/MFA/pretrained_models/dictionary/english_us_arpa.dict, you can now use
                english_us_arpa in place of dictionary paths in mfa commands.


In [ ]:
#| export

from glob import glob
import os
from joblib import Parallel, delayed
from pathlib import Path
from typing import List
from pydub import AudioSegment
import numpy as np
import subprocess
from praatio import textgrid, audio

def chunk_audio(audio: AudioSegment, increment_mills: int = 300, windows: int = 1000) -> List[AudioSegment]:
    sample_rate = audio.frame_count(ms=1000)
    seconds = audio.frame_count()/sample_rate
    def get_segment(e, window):
        return audio[e-windows:e]
    audio_segments = Parallel(n_jobs=4)(delayed(get_segment)(e, windows) for e in np.arange(windows, seconds*1000, increment_mills))
    return audio_segments

def export_audio(segment: AudioSegment, path: str):
    return segment.export(f'{path}.wav', format="wav")

def chunk_selected_audios(audios: List[str], output_path: str) -> None:
    Path(output_path).mkdir(parents=True, exist_ok=True)
    for file in audios:
        audio = AudioSegment.from_wav(file)
        for ix, segment in enumerate(chunk_audio(audio)):
            path = os.path.join(output_path, f"{Path(file).stem}_{ix}.wav")
            export_audio(segment, path)

def align_force_align(corpus_directory: str, output_directory: str):
    dictionary = 'english_us_arpa'
    acoustic_model = 'english_us_arpa'
    return subprocess.run(["mfa", "align", corpus_directory, dictionary, acoustic_model, output_directory], capture_output=True)

def parse_textgrid_items(items: List[tuple], duration_seconds):
    for ix, (k, v) in enumerate(items):
        if 'phones' in k:
            continue
        for s, e, t in v.entryList:
            start = max(s-0.1, 0)*1000
            end = min(e+0.1, duration_seconds)*1000
            yield start, end, t 

def parse_textgrid(corpus_directory: str, output_directory: str):
    grids = glob(os.path.join(output_directory, '*.TextGrid'))
    assert len(grids) == 1, f"{grids}"
    grid = grids[0]
    name = Path(grid).stem
    tg = textgrid.openTextgrid(grid, includeEmptyIntervals=False)
    audio = AudioSegment.from_wav(os.path.join(corpus_directory, f"{name}.wav"))
    parsed = [(s, e, t) for s, e, t in parse_textgrid_items(tg.tierDict.items(), audio.duration_seconds)]
    audios = [audio[start:end] for start, end, _ in parsed]
    transcriptions = [trans for _, _, trans in parsed]
    return audios, transcriptions

In [ ]:
#| eval: false

corpus_directory = 'test/'
output_directory = 'test/output/'
print(align_force_align(corpus_directory, output_directory))
parse_textgrid(corpus_directory, output_directory)


CompletedProcess(args=['mfa', 'align', 'test/', 'english_us_arpa', 'english_us_arpa', 'test/output/'], returncode=0, stdout=b'WARNING - The previous run had a different configuration than the current, which may cause issues. Please see the log\n                   for details or use --clean flag if issues are encountered.\nINFO - Setting up corpus information...\nINFO - Found 1 speaker across 1 file, average number of utterances per speaker: 1.0\nINFO - Initializing multiprocessing jobs...\nWARNING - Number of jobs was specified as 3, but due to only having 1 speakers, MFA will only use 1 jobs.\nINFO - Creating corpus split with features...\nINFO - Alignment already done, skipping.\nINFO - Exporting TextGrids to test/output...\nINFO - Finished exporting TextGrids to test/output!\nINFO - Done! Everything took 6.614829778671265 seconds\n', stderr=b'\r  0%|          | 0/1 [00:00<?, ?it/s]\r100%|\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88\xe2\x96\x88

([<pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>,
  <pydub.audio_segment.AudioSegment>],
 ['ok', 'can', 'i', 'get', 'that', 'please', 'ok', 'ah', 'with', 'a'])

In [ ]:
#| export

import shutil
from typing import List
from collections import Counter
from transformers import AutoFeatureExtractor
from transformers import Wav2Vec2ForXVector, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
import torch
from numpy.typing import NDArray
from pydantic import BaseModel
from io import BytesIO
from glob import glob
from random import choices

class AudioArray(BaseModel):
    array: np.ndarray
    class Config:
        arbitrary_types_allowed = True

class AudioProcessor(object):

    def __init__(self, model_checkpoint: str) -> None:
        self.model_checkpoint = model_checkpoint
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(self.model_checkpoint)
    
    def preprocess_function(self, audio_arrays: List[AudioArray], max_duration: float = 1.0):
        arrays = [a.array for a in audio_arrays]
        inputs = self.feature_extractor(
            arrays,
            sampling_rate=self.feature_extractor.sampling_rate, 
            max_length=int(self.feature_extractor.sampling_rate * max_duration),
            truncation=True,
            return_tensors="pt",
            padding=True
            )
        return inputs
    
    def parse_raw_audio(self, raw_data: bytes, sample_width: int, channels: int, frame_rate: int) -> List[AudioSegment]:
        audio = AudioSegment.from_raw(
            BytesIO(raw_data), 
            sample_width=sample_width,
            channels=channels,
            frame_rate=frame_rate
        )
        return [segment for segment in chunk_audio(audio)]
    
    def encode_raw_audio(self, raw_data: bytes, sample_width: int, channels: int, frame_rate: int):
        parsed = self.parse_raw_audio(raw_data, sample_width, channels, frame_rate)
        parsed = [{'array': np.array(p.get_array_of_samples(), dtype=float)} for p in parsed]
        parsed = [AudioArray.parse_obj(p) for p in parsed]
        return self.preprocess_function(parsed)
    
    def parse_aligned_audio(self, corpus_directory: str):
        output_directory = os.path.join(Path.home(), '.cache/mfa/')
        mfa_output = align_force_align(corpus_directory, output_directory)
        audios, transcriptions = parse_textgrid(corpus_directory, output_directory)
        return audios, transcriptions
    
    def encode_aligned_audio(self, corpus_directory: str):
        parsed, self.transcriptions = self.parse_aligned_audio(corpus_directory)
        parsed = [{'array': np.array(p.get_array_of_samples(), dtype=float)} for p in parsed]
        parsed = [AudioArray.parse_obj(p) for p in parsed]
        return self.preprocess_function(parsed)
    



model_checkpoint = 'data/panda/wav2vec2-base-finetuned-xvector/best_checkpoint/'
audio_processor = AudioProcessor(model_checkpoint)

files = glob(f'{Path.home()}/.cache/panda/audios/*.wav')
file = choices(files, k=1)[0]

AudioSegment.from_wav(file)

In [ ]:
#| eval: false


from IPython.display import display, Audio


with open(file, "rb") as f:
    audio_bytes = bytearray()
    while (byte := f.read(1)):
        audio_bytes.extend(byte)
encoded_data = audio_processor.encode_raw_audio(raw_data=audio_bytes, sample_width=2, channels=2, frame_rate=16000)

for ix, arr in enumerate(encoded_data['input_values']):
    display(Audio(arr.numpy(), rate=32000))


In [ ]:
#| eval: false

encoded_data = audio_processor.encode_aligned_audio('test/')
for ix, arr in enumerate(encoded_data['input_values']):
    print(f"{audio_processor.transcriptions[ix]}:")
    display(Audio(arr.numpy(), rate=32000))


ok:


can:


i:


get:


that:


please:


ok:


ah:


with:


a:


In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()